In [ ]:
import tensorflow as tf
from tensorflow.keras import regularizers, initializers, Sequential
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, Flatten, Dense

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
train_dir = "/kaggle/input/gender-detection-and-classification-image-dataset/test"
test_dir = "/kaggle/input/gender-detection-and-classification-image-dataset/test"

In [ ]:
batch_size = 32
img_size = 224

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                      batch_size=batch_size,
                                                      image_size=(img_size, img_size))

test_ds = tf.keras.utils.image_dataset_from_directory(test_dir,
                                                     batch_size=batch_size,
                                                     image_size=(img_size, img_size))

In [ ]:
class_names = ['man', 'woman']
class_names

In [ ]:
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.prefetch(tf.data.AUTOTUNE)

In [ ]:
for image, label in train_ds.take(1):
    plt.figure(figsize=(12, 10))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        plt.title(class_names[label[i]])
        plt.axis("off")

## Data Augmentation

In [ ]:
data_augmentation = Sequential([
    tf.keras.layers.Resizing(180, 180),
    tf.keras.layers.RandomFlip("horizontal_and_vertical")
])

## Visualize Augmented Images

In [ ]:
for image, label in train_ds.take(1):
    plt.figure(figsize=(12, 10))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        augmented_image = data_augmentation(image)
        plt.imshow(augmented_image[i].numpy().astype("uint8"))
        plt.title(class_names[label[i]])
        plt.axis("off")

## Model Creation

In [ ]:
regularizer = regularizers.l1(0.001)
initializer = initializers.HeNormal(seed=20)

model = Sequential([
    data_augmentation,
    Conv2D(64, 3, activation="relu", kernel_initializer=initializer, kernel_regularizer=regularizer),
    MaxPooling2D((2, 2)),
    Conv2D(32, 3, activation="relu", kernel_initializer=initializer, kernel_regularizer=regularizer),
    MaxPooling2D((2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(10, activation="softmax")
])

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=20)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label='loss')
plt.legend()

In [ ]:
from sklearn.metrics import f1_score

## Test on test dataset

In [ ]:
for image, label in test_ds.take(1):
    plt.figure(figsize=(16, 18))
    for i in range(16):
        plt.subplot(8, 2, i+1)
        plt.imshow(image[i].numpy().astype("uint8"))
        pred = model.predict(tf.expand_dims(image[i], axis=0))
        prediction = np.argmax(pred, axis=1)
        print(prediction)
        plt.title(f"Original Label: {class_names[label[i]]} -- Predicted label: {class_names[int(prediction)]}")
        plt.axis("off")

In [ ]:
model.evaluate(test_ds)

## Save model

In [ ]:
model.save("Gender_classification.keras")